In [1]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
from pyproj import Transformer, CRS
from shapely.geometry import Point

In [2]:
network_root_path = os.path.join(os.getcwd(), 'UKM_PrimarySubstation_Data')
# load the fundamental dataset
UKM_PS_df = pd.read_csv(os.path.join(os.path.dirname(network_root_path), "UKM_PS_data.csv"))
UKM_PS_df

,PS Name,Firm Capacity (MVA),Demand (MVA),PF,"Geo(Long,Lat)",GSP,DNO
0,albion st,22.863,11.720532,0.998712,"-2.4950937454916797,53.73015403195597",ROCHDALE,ENW
1,alderley & chelford,28.310,17.003134,0.975600,"-2.225911796847622,53.30796142675533",SOUTH MANCHESTER,ENW
2,alston,1.700,1.531605,0.985752,"-2.4352546655219385,54.81256830516162",HUTTON,ENW
3,ambleside,17.800,9.192532,0.993175,"-2.963194161426022,54.42314157846089",HUTTON,ENW
4,ancoats north t11 & t12,22.863,17.347102,0.992857,"-2.2271891138444646,53.485968874301655",WHITEGATE,ENW
...,...,...,...,...,...,...,...
4200,Wribbenhall 33/11kv,23.000,14.720000,NaN,"-2.3076,52.3853",Bishops Wood 132kV,WPD
4201,Yelverton,10.000,5.850000,NaN,"-4.0812,50.4948",Abham_Exeter _Landulph,WPD
4202,Ynys Street,13.600,8.160000,NaN,"-3.7753,51.5971",Swansea North 132 kV,WPD
4203,Ynysfeio,10.000,6.940000,NaN,"-3.5255,51.6697",Swansea North 132 kV,WPD


## We first find the corresponding LA for the PSs 
LA: Local Authority
PS: Primary Substation

In [3]:
root_path = os.getcwd()
uk_la_map_path = os.path.join(root_path, "Geo_data", "LAD_DEC_2022_UK_BFC.shp")
eng_region_map_df = gpd.read_file(os.path.join(root_path, "Geo_data", "Regions_(December_2022)_EN_BFC.shp"))
uk_la_map_df = gpd.read_file(uk_la_map_path)
# these codes can be found in the gov web
Scot_la_code_list = ['S12000005','S12000006','S12000008','S12000010','S12000011','S12000013','S12000014','S12000017',
                     'S12000018','S12000019','S12000020','S12000021','S12000023','S12000026','S12000027','S12000028',
                     'S12000029','S12000030','S12000033','S12000034','S12000035','S12000036','S12000038','S12000039',
                     'S12000040','S12000041','S12000042','S12000045','S12000047','S12000048','S12000049','S12000050']
Wales_la_code_list = ['W06000001', 'W06000002', 'W06000003', 'W06000004', 'W06000005', 'W06000006', 'W06000023', 'W06000008', 'W06000009', 'W06000010',
                      'W06000011', 'W06000012', 'W06000013', 'W06000014', 'W06000015', 'W06000016', 'W06000024', 'W06000018', 'W06000019', 'W06000020',
                      'W06000021', 'W06000022']

ScotWales_la_code_list = Scot_la_code_list + Wales_la_code_list
ScotWales_la_map_df = uk_la_map_df[uk_la_map_df["LAD22CD"].isin(ScotWales_la_code_list)]

map_df = pd.concat([ScotWales_la_map_df.rename(columns={"LAD22CD":"RegID", "LAD22NM":"RegName"}), eng_region_map_df.rename(columns={"RGN22CD":"RegID", "RGN22NM":"RegName"})]).pipe(gpd.GeoDataFrame)

del uk_la_map_df
del ScotWales_la_map_df
del eng_region_map_df

def find_reg(lon, lat, map_df):
    # this function finds the region that contains the point (lon, lat)
    # 4326 is the (long, lat) system while 27700 is the coordinates specified in the .proj file or read from geodf.crs
    src_crs = CRS('EPSG:4326')
    des_crs = CRS('EPSG:27700')
    transformer = Transformer.from_crs(src_crs, des_crs, always_xy=True)
    # Convert the coordinate to the destination projection system
    x2,y2 = transformer.transform(lon, lat)

    point = Point(x2, y2)
    for index, row in map_df.iterrows():
        polygon = row['geometry']
        if polygon.contains(point):
            return row["RegID"], row["RegName"]
        
    return np.nan, np.nan

In [4]:
# find the LA for each PS
from IPython.display import clear_output

RegID_list = []
RegName_list = []
for i, item in enumerate(UKM_PS_df["Geo(Long,Lat)"]):
    # clear output to have a dynamic progress bar
    clear_output(wait=True)
    print(f"processed: {100 * (i+1) / len(UKM_PS_df['Geo(Long,Lat)'])}%")
    lon, lat = item.split(",")
    RegID, RegName = find_reg(lon = lon, lat = lat, map_df = map_df)

    RegID_list.append(RegID)
    RegName_list.append(RegName)
    
UKM_PS_df["RegID"] = RegID_list
UKM_PS_df["RegName"] = RegName_list

processed: 100.0%


In [5]:
# find those with nan LA ID
# the existence of nan is due to the minor inaccuracy of the boundary information.
UKM_PS_df[UKM_PS_df["RegID"].isna()]

,PS Name,Firm Capacity (MVA),Demand (MVA),PF,"Geo(Long,Lat)",GSP,DNO,RegID,RegName
2545,erith,22.30,9.20544,NaN,"0.17513466613484338,51.486604037233285",Littlebrook,UKPN,NaN,NaN
2742,leigh primary,14.50,13.69960,NaN,"0.6605892765537588,51.53632872117139",Rayleigh,UKPN,NaN,NaN
2987,reydon primary,4.80,4.55616,NaN,"1.626197991212215,52.32998881373491",Bramford,UKPN,NaN,NaN
3102,deptford grid,119.60,60.60132,NaN,"-0.01737230934444808,51.479838753369016",New Cross 132kV,UKPN,NaN,NaN
3107,barnes b,28.80,17.49888,NaN,"-0.2545756097115028,51.47250291377867",Wimbledon 3&4 132kV,UKPN,NaN,NaN
3143,north shoreham,16.60,11.52538,NaN,"-0.29449041401571696,50.853413716824306",Bolney,UKPN,NaN,NaN
3508,Ernesettle B & S,2.40,2.09000,NaN,"-4.1873,50.4241",Abham_Exeter _Landulph,WPD,NaN,NaN
3820,Newton Abbot Main,28.86,22.08000,NaN,"-3.5991,50.5338",Abham_Exeter _Landulph,WPD,NaN,NaN


In [6]:
# those with nan LA ID are manually found through google map
UKM_PS_df.loc[UKM_PS_df["PS Name"] == "erith", "RegName"] = "London"
UKM_PS_df.loc[UKM_PS_df["PS Name"] == "erith", "RegID"] = "E12000007"

UKM_PS_df.loc[UKM_PS_df["PS Name"] == "leigh primary", "RegName"] = "East of England"
UKM_PS_df.loc[UKM_PS_df["PS Name"] == "leigh primary", "RegID"] = "E12000006"

UKM_PS_df.loc[UKM_PS_df["PS Name"] == "reydon primary", "RegName"] = "East of England"
UKM_PS_df.loc[UKM_PS_df["PS Name"] == "reydon primary", "RegID"] = "E12000006"

UKM_PS_df.loc[UKM_PS_df["PS Name"] == "deptford grid", "RegName"] = "London"
UKM_PS_df.loc[UKM_PS_df["PS Name"] == "deptford grid", "RegID"] = "E12000007"

UKM_PS_df.loc[UKM_PS_df["PS Name"] == "barnes b", "RegName"] = "London"
UKM_PS_df.loc[UKM_PS_df["PS Name"] == "barnes b", "RegID"] = "E12000007"

UKM_PS_df.loc[UKM_PS_df["PS Name"] == "north shoreham", "RegName"] = "South East"
UKM_PS_df.loc[UKM_PS_df["PS Name"] == "north shoreham", "RegID"] = "E12000008"

UKM_PS_df.loc[UKM_PS_df["PS Name"] == "Ernesettle B & S", "RegName"] = "South West"
UKM_PS_df.loc[UKM_PS_df["PS Name"] == "Ernesettle B & S", "RegID"] = "E12000009"

UKM_PS_df.loc[UKM_PS_df["PS Name"] == "Newton Abbot Main", "RegName"] = "South West"
UKM_PS_df.loc[UKM_PS_df["PS Name"] == "Newton Abbot Main", "RegID"] = "E12000009"

# set all RegID starting with W to W92000004 (Wales)
UKM_PS_df.loc[UKM_PS_df["RegID"].str.startswith("W"), "RegID"] = "W92000004"

## Load Census data

In [7]:
UKM_PS_df_extend = UKM_PS_df.copy()

# load all the census data
# heating proportion
census_root_path = os.path.join(os.getcwd(), "Census_data")
eng_heat_prop_df = pd.read_csv(os.path.join(census_root_path, "2021CensusFuelProportionEnglandWales_MSOA.csv"))[["Middle Layer Super Output Areas Code", "Type of central heating in household (13 categories)", "Observation"]]

# pivot it
eng_heat_prop_df = eng_heat_prop_df.pivot_table(values='Observation', index="Middle Layer Super Output Areas Code", columns='Type of central heating in household (13 categories)').reset_index()

# scotland data needs to be processed
# the loaded data is for the lowest data zone level, which needs to be aggregated to intermediate zone level
# the first row is for the whole scotland, while the last three are comments, so all dropped
scot_heat_prop_df = pd.read_csv(os.path.join(census_root_path, "QS415SC.csv"), header = 3).iloc[1:-3,:]
for i in range(len(scot_heat_prop_df.columns)-1):
    # replace NaN to 0 and convert to numeric
    scot_heat_prop_df[scot_heat_prop_df.columns[i+1]] = pd.to_numeric(scot_heat_prop_df.iloc[:,i+1], errors='coerce').replace(np.nan, 0)

# load the lookup table for aggregating the data zone level data to intermediate zone level
scot_zonelookup_df = pd.read_csv(os.path.join(census_root_path, "DataZone2011lookup.csv"))
# process scot_heat_prop_df so that it becomes data per intermediate zone
scot_zonelookup_dict = {scot_zonelookup_df.iloc[i]["DZ2011_Code"]: scot_zonelookup_df.iloc[i]["IZ2011_Code"] for i in range(len(scot_zonelookup_df))}
scot_heat_prop_df["IZ_code"] = scot_heat_prop_df.iloc[:,0].map(scot_zonelookup_dict)
scot_heat_prop_df.drop(columns = ["Unnamed: 0"], inplace = True)
scot_heat_prop_df = scot_heat_prop_df.groupby(by="IZ_code").sum().reset_index()


### process these census heating data, making them contain:
1. Number of households in different types of heating
2. Number of households in total

In [8]:
# for Scotland
scot_heat_prop_df_processed = scot_heat_prop_df.rename(columns = {"All occupied household spaces":"Total Households"}).copy()
# for England and Wales. Rename for consistency with Scotland data
ew_heat_prop_df_processed = eng_heat_prop_df.rename(columns = {"Electric only": "Electric (including storage heaters) central heating", "Mains gas only": "Gas central heating", "Oil only":"Oil central heating", "Solid fuel only":"Solid fuel (for example wood, coal) central heating", "Middle Layer Super Output Areas Code":"MSOA_code", "Other central heating only": "Other central heating"}).copy()
# calculate the total number of households
ew_heat_prop_df_processed["Total Households"] = ew_heat_prop_df_processed.iloc[:,1:].sum(axis = 1)
# remove the header of index
ew_heat_prop_df_processed.rename_axis(None, axis=1, inplace=True)
ew_heat_prop_df_processed

,MSOA_code,District or communal heat networks only,Does not apply,Electric (including storage heaters) central heating,Gas central heating,No central heating,Oil central heating,Other central heating,Renewable energy only,"Solid fuel (for example wood, coal) central heating",Tank or bottled gas only,Two or more types of central heating (including renewable energy),Two or more types of central heating (not including renewable energy),Wood only,Total Households
0,E02000001,915,0,1937,1323,184,15,145,9,0,29,9,348,1,4915
1,E02000002,42,0,317,2116,63,1,49,3,0,15,13,252,0,2871
2,E02000003,20,0,416,2981,89,4,78,6,0,14,11,310,1,3930
3,E02000004,6,0,82,1958,23,2,31,0,0,21,4,181,0,2308
4,E02000005,23,0,221,2887,69,0,65,1,0,19,26,316,0,3627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7259,W02000424,2,0,67,2397,8,145,8,9,7,56,19,249,3,2970
7260,W02000425,5,0,143,1600,18,577,4,19,16,183,20,272,15,2872
7261,W02000426,2,0,85,3871,32,30,10,4,3,18,12,326,2,4395
7262,W02000427,3,0,110,4015,41,136,22,7,94,53,14,381,14,4890


In [9]:
scot_heat_prop_df_processed

,IZ_code,Total Households,No central heating,Gas central heating,Electric (including storage heaters) central heating,Oil central heating,"Solid fuel (for example wood, coal) central heating",Other central heating,Two or more types of central heating
0,S02001236,2143.0,27.0,1683.0,264.0,108.0,10.0,7.0,44.0
1,S02001237,1428.0,12.0,1218.0,48.0,117.0,1.0,10.0,22.0
2,S02001238,2291.0,13.0,2033.0,157.0,57.0,1.0,2.0,28.0
3,S02001239,2439.0,39.0,1995.0,334.0,8.0,2.0,7.0,54.0
4,S02001240,2193.0,64.0,1894.0,192.0,3.0,0.0,4.0,36.0
...,...,...,...,...,...,...,...,...,...
1274,S02002510,2012.0,35.0,1634.0,131.0,117.0,36.0,13.0,46.0
1275,S02002511,1780.0,23.0,1574.0,125.0,10.0,10.0,2.0,36.0
1276,S02002512,2485.0,28.0,2192.0,154.0,48.0,9.0,7.0,47.0
1277,S02002513,1479.0,15.0,1372.0,61.0,4.0,3.0,5.0,19.0


## Now we can estimate the number of households connected to each PS

In [10]:
# we first need to know the total number of households for each region.
# For Scotland, the term region is for local authority
scot_houseno_df = scot_heat_prop_df.iloc[:,:2].copy()
# the number of households per LA for scotland can be aggregated from the scot_heat_prop_df at the IZ level
# get the dict maps IZ to LA
scot_zonelookup_dict = {scot_zonelookup_df.iloc[i]["IZ2011_Code"]: scot_zonelookup_df.iloc[i]["LA_Code"] for i in range(len(scot_zonelookup_df))}
scot_houseno_df["LA_code"] = scot_houseno_df.iloc[:,0].map(scot_zonelookup_dict)
# get the number of households per LA
scot_houseno_df = scot_houseno_df.groupby(by="LA_code").sum(numeric_only = True).reset_index()
scot_la2householdno_dict = {scot_houseno_df.iloc[i]["LA_code"]: scot_houseno_df.iloc[i]["All occupied household spaces"] for i in range(len(scot_houseno_df))}

# for England and Wales, do the same thing
ew_houseno_df = ew_heat_prop_df_processed[["MSOA_code", "Total Households"]].copy()
# load the dict
ew_msoalookup_df = pd.read_csv(os.path.join(census_root_path, "MSOA_2021lookup.csv"))
# get the dict maps MSOA to region
ew_msoaookup_dict = {ew_msoalookup_df.iloc[i]["MSOA21CD"]: ew_msoalookup_df.iloc[i]["RGN22CD"] for i in range(len(ew_msoalookup_df))}
# sum over the same RegID
ew_houseno_df["RegID"] = ew_houseno_df.iloc[:,0].map(ew_msoaookup_dict)
ew_houseno_df = ew_houseno_df.groupby(by="RegID").sum(numeric_only = True).reset_index()
ew_reg2householdno_dict = {ew_houseno_df.iloc[i]["RegID"]: ew_houseno_df.iloc[i]["Total Households"] for i in range(len(ew_houseno_df))}

ew_reg2householdno_dict

{'E12000001': 1175659,
 'E12000002': 3153359,
 'E12000003': 2330688,
 'E12000004': 2037356,
 'E12000005': 2429388,
 'E12000006': 2628799,
 'E12000007': 3423746,
 'E12000008': 3807978,
 'E12000009': 2448892,
 'W92000004': 1347094}

In [11]:
# based on these dicts, we can estimate the number of households connected to each PS
la2householdno_dict = {**scot_la2householdno_dict, **ew_reg2householdno_dict}
##
PS_household_no_list = []
for i in range(len(UKM_PS_df_extend)):
    reg_code = UKM_PS_df_extend.iloc[i]['RegID']
    demand = UKM_PS_df_extend.iloc[i]['Demand (MVA)']
    reg_total_demand = UKM_PS_df_extend[UKM_PS_df_extend["RegID"] == reg_code]["Demand (MVA)"].sum()
    prop_by_demand = demand / reg_total_demand
    household_no = prop_by_demand * la2householdno_dict[reg_code]

    PS_household_no_list.append(household_no)

UKM_PS_df_extend["Household no"] = PS_household_no_list

In [12]:
UKM_PS_df_extend

,PS Name,Firm Capacity (MVA),Demand (MVA),PF,"Geo(Long,Lat)",GSP,DNO,RegID,RegName,Household no
0,albion st,22.863,11.720532,0.998712,"-2.4950937454916797,53.73015403195597",ROCHDALE,ENW,E12000002,North West,6445.710749
1,alderley & chelford,28.310,17.003134,0.975600,"-2.225911796847622,53.30796142675533",SOUTH MANCHESTER,ENW,E12000002,North West,9350.879498
2,alston,1.700,1.531605,0.985752,"-2.4352546655219385,54.81256830516162",HUTTON,ENW,E12000002,North West,842.306781
3,ambleside,17.800,9.192532,0.993175,"-2.963194161426022,54.42314157846089",HUTTON,ENW,E12000002,North West,5055.436163
4,ancoats north t11 & t12,22.863,17.347102,0.992857,"-2.2271891138444646,53.485968874301655",WHITEGATE,ENW,E12000002,North West,9540.045201
...,...,...,...,...,...,...,...,...,...,...
4200,Wribbenhall 33/11kv,23.000,14.720000,NaN,"-2.3076,52.3853",Bishops Wood 132kV,WPD,E12000005,West Midlands,10531.051992
4201,Yelverton,10.000,5.850000,NaN,"-4.0812,50.4948",Abham_Exeter _Landulph,WPD,E12000009,South West,3010.314816
4202,Ynys Street,13.600,8.160000,NaN,"-3.7753,51.5971",Swansea North 132 kV,WPD,W92000004,Neath Port Talbot,4712.338871
4203,Ynysfeio,10.000,6.940000,NaN,"-3.5255,51.6697",Swansea North 132 kV,WPD,W92000004,Rhondda Cynon Taf,4007.798011


In [13]:
flag = round(UKM_PS_df_extend["Household no"].sum()) == round(sum(scot_la2householdno_dict.values()) + sum(ew_reg2householdno_dict.values()))
# verify if the summation of the estimated number of households connected to each PS matches the summation of the total number of households in the UKM
if flag:
    print("The estimation matches the summation")
else:
    print("The estimation does not match the summation")

The estimation matches the summation


In [14]:
round(sum(scot_la2householdno_dict.values()) + sum(ew_reg2householdno_dict.values()))

27155736

In [15]:
# display the mean and median number of connected households for Scotland PS (RegID starging with S)
# and those for England and Wales PS (RegID starting with E or W)
scot_PS_household_no_mean = UKM_PS_df_extend[UKM_PS_df_extend["RegID"].str.startswith("S")]["Household no"].mean()
scot_PS_household_no_median = UKM_PS_df_extend[UKM_PS_df_extend["RegID"].str.startswith("S")]["Household no"].median()
ew_PS_household_no_mean = UKM_PS_df_extend[UKM_PS_df_extend["RegID"].str.startswith("E") | UKM_PS_df_extend["RegID"].str.startswith("W")]["Household no"].mean()
ew_PS_household_no_median = UKM_PS_df_extend[UKM_PS_df_extend["RegID"].str.startswith("E") | UKM_PS_df_extend["RegID"].str.startswith("W")]["Household no"].median()
print(f"The mean number of households for Scotland PS is {scot_PS_household_no_mean}, while the median is {scot_PS_household_no_median}")
print(f"The mean number of households for England and Wales PS is {ew_PS_household_no_mean}, while the median is {ew_PS_household_no_median}")

The mean number of households for Scotland PS is 3206.4554054054056, while the median is 2877.7998450417026
The mean number of households for England and Wales PS is 7152.36911976912, while the median is 6125.804675927737


In [16]:
# display the mean and median number of households for Scotland IZ in scot_heat_prop_df_processed
# and those for England and Wales MSOA in ew_heat_prop_df_processed
scot_IZ_household_no_mean = scot_heat_prop_df_processed["Total Households"].mean()
scot_IZ_household_no_median = scot_heat_prop_df_processed["Total Households"].median()
ew_MSOA_household_no_mean = ew_heat_prop_df_processed["Total Households"].mean()
ew_MSOA_household_no_median = ew_heat_prop_df_processed["Total Households"].median()
print(f"The mean number of households for Scotland IZ is {scot_IZ_household_no_mean}, while the median is {scot_IZ_household_no_median}")
print(f"The mean number of households for England and Wales MSOA is {ew_MSOA_household_no_mean}, while the median is {ew_MSOA_household_no_median}")

The mean number of households for Scotland IZ is 1855.1813917122752, while the median is 1802.0
The mean number of households for England and Wales MSOA is 3411.7509636563877, while the median is 3302.5


## Now we can link the PSs to other census data 
The mean and median number of households for an MSOA are 3412 and 3202. The mean and median household numbers of an IZ are 1855 and 1802. A PS can roughly support two MSOAs or two IZs in England or Scotland. Therefore, we may estimate the household heating attributes for each PS by finding the two nearest MSOA or IZ for each PS and then combine the information.

### Step 1 get the two nearest MSOA or IZ for each PS by KNN

In [17]:
# this cell finds the nearest intermediate zone (IZ) for Scotland, or MSOA for England and Wales for each PS 
from geopy import distance
from sklearn.neighbors import NearestNeighbors

def UK_to_lonlat(east, north):
    # 4326 is the long, lat system while 27700 is the coordinates specified in the .proj file or read from geodf.crs
    # this convert the location under UK map coordinate to lat, lon format
    des_crs = CRS('EPSG:4326')
    src_crs = CRS('EPSG:27700')
    transformer = Transformer.from_crs(src_crs, des_crs, always_xy=True)
    # Convert the coordinate to the destination projection system
    lon, lat = transformer.transform(east, north)
    return lat, lon

k = 2 # the 2 nearest zones
def nearest_zone(loc_array, zone_loc_array, k=k):
    nbrs = NearestNeighbors(n_neighbors=k, metric=lambda x, y:distance.great_circle(x, y).km).fit(zone_loc_array)
    distances, indices = nbrs.kneighbors(loc_array)
    return indices

def geostr2array(geostr):
    return np.array(geostr.str.split(",").tolist()).astype("float")

### load df storing the zone id and locations of the population centroids
root_path = os.getcwd()
# the zone_geo_df stores the zone id and its weighted population centroids for England and Scotland and Wales
zone_geo_df = pd.concat((gpd.read_file(os.path.join(root_path, "Geo_data", "ScotInterZ2011", "SG_IntermediateZone_Cent_2011.shp"))[["InterZone", "geometry"]].rename(columns = {"InterZone":"ZoneID"}), gpd.read_file(os.path.join(root_path, "Geo_data", "EngWalesMSOA_2021Dec", "MLSOA_(Dec_2021)_PWC_in_England_and_Wales.shp"))[["MSOA21CD", "geometry"]].rename(columns = {"MSOA21CD":"ZoneID"})), axis = 0).reset_index(drop=True)

lonlat_list = []
lat, lon = UK_to_lonlat(east = zone_geo_df["geometry"].x, north = zone_geo_df["geometry"].y)

lat = pd.DataFrame(lat)
lon = pd.DataFrame(lon)

zone_geo_df["Geo(Long,Lat)"] = lon.astype("str")+','+lat.astype("str")
zone_geo_df.drop(columns=["geometry"], inplace=True)
zone_geo_df

,ZoneID,"Geo(Long,Lat)"
0,S02001236,"-2.268110228575192,57.09985072971953"
1,S02001237,"-2.2302460609539563,57.10766050782388"
2,S02001238,"-2.191231202546192,57.11634991250586"
3,S02001239,"-2.134120238399147,57.12258648772218"
4,S02001240,"-2.118878345318312,57.13342645999975"
...,...,...
8538,E02007042,"-1.4265155562151268,54.34008565351061"
8539,E02001686,"-1.7253957290254134,54.959494966987506"
8540,E02004011,"-2.74303745278021,54.66619873935894"
8541,E02003245,"-0.2760936797319316,52.592223233110445"


In [18]:
# now find the two nearest zones for each PS
nearest_zone_idx = nearest_zone(geostr2array(UKM_PS_df_extend["Geo(Long,Lat)"]), geostr2array(zone_geo_df["Geo(Long,Lat)"]))
nearest_zone_code = zone_geo_df["ZoneID"].values[nearest_zone_idx]
UKM_PS_df_extend["Zone"] = list(nearest_zone_code)
for i in range(k):
    UKM_PS_df_extend[f"NearestZone{i}"] = UKM_PS_df_extend["Zone"].str[i]

UKM_PS_df_extend.drop(columns=["Zone"], inplace=True)

In [19]:
UKM_PS_df_extend

,PS Name,Firm Capacity (MVA),Demand (MVA),PF,"Geo(Long,Lat)",GSP,DNO,RegID,RegName,Household no,NearestZone0,NearestZone1
0,albion st,22.863,11.720532,0.998712,"-2.4950937454916797,53.73015403195597",ROCHDALE,ENW,E12000002,North West,6445.710749,E02002623,E02002627
1,alderley & chelford,28.310,17.003134,0.975600,"-2.225911796847622,53.30796142675533",SOUTH MANCHESTER,ENW,E12000002,North West,9350.879498,E02003864,E02003858
2,alston,1.700,1.531605,0.985752,"-2.4352546655219385,54.81256830516162",HUTTON,ENW,E12000002,North West,842.306781,E02005728,E02004008
3,ambleside,17.800,9.192532,0.993175,"-2.963194161426022,54.42314157846089",HUTTON,ENW,E12000002,North West,5055.436163,E02004015,E02004016
4,ancoats north t11 & t12,22.863,17.347102,0.992857,"-2.2271891138444646,53.485968874301655",WHITEGATE,ENW,E12000002,North West,9540.045201,E02006902,E02006912
...,...,...,...,...,...,...,...,...,...,...,...,...
4200,Wribbenhall 33/11kv,23.000,14.720000,NaN,"-2.3076,52.3853",Bishops Wood 132kV,WPD,E12000005,West Midlands,10531.051992,E02006771,E02006777
4201,Yelverton,10.000,5.850000,NaN,"-4.0812,50.4948",Abham_Exeter _Landulph,WPD,E12000009,South West,3010.314816,E02004234,E02004235
4202,Ynys Street,13.600,8.160000,NaN,"-3.7753,51.5971",Swansea North 132 kV,WPD,W92000004,Neath Port Talbot,4712.338871,W02000216,W02000215
4203,Ynysfeio,10.000,6.940000,NaN,"-3.5255,51.6697",Swansea North 132 kV,WPD,W92000004,Rhondda Cynon Taf,4007.798011,W02000258,W02000261


### Step 2 combine information

In [20]:
# first combine the Scotland, England, and Wales Census heating data
processed_heat_prop_df = pd.concat([scot_heat_prop_df_processed.rename(columns = {"IZ_code":"ZoneID"}), ew_heat_prop_df_processed.rename(columns = {"MSOA_code":"ZoneID"})], axis=0).reset_index(drop=True)

# enumerate atributes to be included
for atr in processed_heat_prop_df.columns[2:]:
    atr_res = 0
    tot_number = 0
    for i in range(k):
        atr_res += UKM_PS_df_extend[f"NearestZone{i}"].map(processed_heat_prop_df.set_index("ZoneID")[atr])
        tot_number += UKM_PS_df_extend[f"NearestZone{i}"].map(processed_heat_prop_df.set_index("ZoneID")["Total Households"])
    # this propotion is the weighted average of the attributes of the two nearest zones
    atr_prop = atr_res / tot_number
    UKM_PS_df_extend[atr] = atr_prop * UKM_PS_df_extend["Household no"]
    
    # To ensure consistency, calibrate so that the summed PS heating attribute match the summed census heating attribute in the UKM for each attribute
    processed_heat_prop_df_sum = processed_heat_prop_df[atr].sum()
    UKM_PS_df_extend_sum = UKM_PS_df_extend[atr].sum()
    UKM_PS_df_extend[atr] = UKM_PS_df_extend[atr] * processed_heat_prop_df_sum / UKM_PS_df_extend_sum

In [21]:
# verification, sum over all the atr in processed_heat_prop_df and compare the summation in UKM_PS_df_extend
processed_heat_prop_df_sum = processed_heat_prop_df.iloc[:,2:].sum(axis=0)
UKM_PS_df_extend_sum = UKM_PS_df_extend.iloc[:, -len(processed_heat_prop_df.columns[2:]):].sum(axis=0)

flag = np.allclose(processed_heat_prop_df_sum, UKM_PS_df_extend_sum)
# print if the estimation match the summation
if flag:
    print("The estimation matches the summation")
else:
    print("The estimation does not match the summation")

The estimation matches the summation


In [22]:
UKM_PS_df_extend.min()

C:\Users\Zhou_Ming\AppData\Local\Temp\ipykernel_113868\1032193298.py:1: FutureWarning: The default value of numeric_only in DataFrame.min is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  UKM_PS_df_extend.min()


PS Name                                                                                                 Aaronsons
Firm Capacity (MVA)                                                                                           0.0
Demand (MVA)                                                                                                  0.0
PF                                                                                                      -0.992422
Geo(Long,Lat)                                                            -0.0016544782309627724,51.45862753684987
DNO                                                                                                           ENW
RegID                                                                                                   E12000001
RegName                                                                                             Aberdeen City
Household no                                                                            

The "Does not apply" attribute is all NaN, as all the Does not apply entries in the Census data are 0. Therefore we can drop this columns

In [23]:
UKM_PS_df_extend.drop(columns = "Does not apply", inplace = True)
# save UKM_PS_df_extend
UKM_PS_df_extend.to_csv(os.path.join(os.path.dirname(network_root_path), "UKM_PS_data_extend.csv"), index = False)
UKM_PS_df_extend    

,PS Name,Firm Capacity (MVA),Demand (MVA),PF,"Geo(Long,Lat)",GSP,DNO,RegID,RegName,Household no,...,Oil central heating,"Solid fuel (for example wood, coal) central heating",Other central heating,Two or more types of central heating,District or communal heat networks only,Renewable energy only,Tank or bottled gas only,Two or more types of central heating (including renewable energy),Two or more types of central heating (not including renewable energy),Wood only
0,albion st,22.863,11.720532,0.998712,"-2.4950937454916797,53.73015403195597",ROCHDALE,ENW,E12000002,North West,6445.710749,...,13.826672,6.392052,81.273088,NaN,25.844789,8.601603,22.806265,16.321717,627.055880,1.832096
1,alderley & chelford,28.310,17.003134,0.975600,"-2.225911796847622,53.30796142675533",SOUTH MANCHESTER,ENW,E12000002,North West,9350.879498,...,312.522343,5.786863,49.232524,NaN,22.422934,42.245650,125.073193,43.551502,784.517529,13.061761
2,alston,1.700,1.531605,0.985752,"-2.4352546655219385,54.81256830516162",HUTTON,ENW,E12000002,North West,842.306781,...,257.121580,42.046245,3.722035,NaN,2.053192,16.072124,41.852626,12.898245,136.833255,18.950289
3,ambleside,17.800,9.192532,0.993175,"-2.963194161426022,54.42314157846089",HUTTON,ENW,E12000002,North West,5055.436163,...,305.965367,13.738314,20.978566,NaN,17.358657,37.205608,83.305441,46.042435,629.628917,36.177518
4,ancoats north t11 & t12,22.863,17.347102,0.992857,"-2.2271891138444646,53.485968874301655",WHITEGATE,ENW,E12000002,North West,9540.045201,...,11.107747,0.000000,88.335239,NaN,140.888930,20.730461,35.938447,38.301303,307.581056,2.207740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,Wribbenhall 33/11kv,23.000,14.720000,NaN,"-2.3076,52.3853",Bishops Wood 132kV,WPD,E12000005,West Midlands,10531.051992,...,1183.416478,82.857096,46.148482,NaN,3.563974,68.085217,1112.591572,71.313260,1364.989654,74.277481
4201,Yelverton,10.000,5.850000,NaN,"-4.0812,50.4948",Abham_Exeter _Landulph,WPD,E12000009,South West,3010.314816,...,601.591459,11.014836,9.105447,NaN,0.000000,42.594758,121.957101,44.720666,451.572196,38.383484
4202,Ynys Street,13.600,8.160000,NaN,"-3.7753,51.5971",Swansea North 132 kV,WPD,W92000004,Neath Port Talbot,4712.338871,...,14.220687,7.379210,41.560411,NaN,12.205691,3.159543,23.198410,12.621680,360.027886,4.037797
4203,Ynysfeio,10.000,6.940000,NaN,"-3.5255,51.6697",Swansea North 132 kV,WPD,W92000004,Rhondda Cynon Taf,4007.798011,...,5.821549,15.138531,20.757866,NaN,0.485787,5.432398,16.619358,38.655321,273.372823,1.446340


In [24]:
UKM_PS_df_extend["Gas central heating"] / UKM_PS_df_extend["Household no"]

0       0.827133
1       0.756705
2       0.336426
3       0.656443
4       0.152488
          ...   
4200    0.576709
4201    0.511566
4202    0.860373
4203    0.896920
4204    0.583733
Length: 4205, dtype: float64

In [25]:
UKM_PS_df_extend["Gas central heating"].sum() / UKM_PS_df_extend["Household no"].sum()

0.7387065112136898

In [26]:
processed_heat_prop_df["Gas central heating"].sum() / processed_heat_prop_df["Total Households"].sum()

0.7387065112136898

In [27]:
UKM_PS_df_extend["Household no"].sum(), processed_heat_prop_df["Total Households"].sum()

(27155736.0, 27155736.0)

In [28]:
processed_heat_prop_df

,ZoneID,Total Households,No central heating,Gas central heating,Electric (including storage heaters) central heating,Oil central heating,"Solid fuel (for example wood, coal) central heating",Other central heating,Two or more types of central heating,District or communal heat networks only,Does not apply,Renewable energy only,Tank or bottled gas only,Two or more types of central heating (including renewable energy),Two or more types of central heating (not including renewable energy),Wood only
0,S02001236,2143.0,27.0,1683.0,264.0,108.0,10.0,7.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S02001237,1428.0,12.0,1218.0,48.0,117.0,1.0,10.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,S02001238,2291.0,13.0,2033.0,157.0,57.0,1.0,2.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,S02001239,2439.0,39.0,1995.0,334.0,8.0,2.0,7.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,S02001240,2193.0,64.0,1894.0,192.0,3.0,0.0,4.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8538,W02000424,2970.0,8.0,2397.0,67.0,145.0,7.0,8.0,NaN,2.0,0.0,9.0,56.0,19.0,249.0,3.0
8539,W02000425,2872.0,18.0,1600.0,143.0,577.0,16.0,4.0,NaN,5.0,0.0,19.0,183.0,20.0,272.0,15.0
8540,W02000426,4395.0,32.0,3871.0,85.0,30.0,3.0,10.0,NaN,2.0,0.0,4.0,18.0,12.0,326.0,2.0
8541,W02000427,4890.0,41.0,4015.0,110.0,136.0,94.0,22.0,NaN,3.0,0.0,7.0,53.0,14.0,381.0,14.0


In [29]:
processed_heat_prop_df.iloc[:,2:].sum(axis=0).sum()

27155736.0